In [4]:
import pandas as pd
import time

In [5]:
import urllib.parse
import httplib2
import json

def getQueryData(sql):
    http = httplib2.Http()
    url = 'http://ip:8004/ConfComputer/sql/select.do'   
    body = {'pwd': 'eship001~', 'sql': sql}
    headers = {'Content-type': 'application/x-www-form-urlencoded'}
    response, content = http.request(url, 'POST', headers=headers, body=urllib.parse.urlencode(body))
    return content

data=getQueryData('show databases')
data

b'[{"result":"android"},{"result":"default"},{"result":"test"},{"result":"web_site"}]'

In [6]:
app_version_all=['5.0.0','5.0.0(F)','5.1.0(F)','5.1.5(F)','5.2.0','5.3.0','5.4.0','5.5.0','5.5.1','5.6.0','5.6.1']

In [7]:
def to_tuple_sql(app_version):
    if type(app_version) == str:
        temp=list([app_version])
        temp.append('all')
        return tuple(temp)
    if type(app_version) == list:
        return tuple(app_version)

In [8]:
def active_num(want,startdate='2016-07-04',enddate='2016-07-31',
               app_version_all=['5.0.0','5.0.0(F)','5.1.0(F)','5.1.5(F)','5.2.0','5.3.0','5.4.0','5.5.0','5.5.1','5.6.0','5.6.1']):
    '''
    want='dt,app_version'
    db='android'
    startdate='2016-07-04'
    enddate='2016-07-31'
    '''
    db='android'
    table='android_log_data'
    
    sql='use {};'.format(db)
    sql+="select {},count(distinct tracking_id) as active_num from {}".format(want,table)
    sql+=" where dt >='{}' and dt <='{}'".format(startdate,enddate)
    sql+=" and app_version in {}".format(to_tuple_sql(app_version_all))
    sql+=" group by {}".format(want)
    sql+=' limit 10000'
    
    return sql

In [1]:
def new_num(want,startdate='2016-07-04',enddate='2016-07-31',
               app_version_all=['5.0.0','5.0.0(F)','5.1.0(F)','5.1.5(F)','5.2.0','5.3.0','5.4.0','5.5.0','5.5.1','5.6.0','5.6.1']):
    '''
    want='dt,app_version'
    db='android'
    startdate='2016-07-04'
    enddate='2016-07-31'
    '''
    db='android'
    table='android_log_data'
    
    sql='use {};'.format(db)
    sql+="select {},count(install_event) as active_num from {}".format(want,table)
    sql+=" where dt >='{}' and dt <='{}'".format(startdate,enddate)
    #sql+=" and app_version in {}".format(to_tuple_sql(app_version_all))
    sql+=" and app_version like '%.%.%' and length(app_version)<10"
    sql+=" group by {}".format(want)
    sql+=' limit 10000'
    
    return sql

In [10]:
want='dt,channels'
startdate='2016-01-01'
enddate='2016-07-31'

path=want.replace(',','+')+'_'+startdate.replace('-','')+'-'+enddate.replace('-','')
path

'dt+channels_20160101-20160731'

In [11]:
sql=new_num(want,startdate=startdate,enddate=enddate)

In [12]:
sql

"use android;select dt,channels,count(install_event) as active_num from android_log_data where dt >='2016-01-01' and dt <='2016-07-31' and app_version in ('5.0.0', '5.0.0(F)', '5.1.0(F)', '5.1.5(F)', '5.2.0', '5.3.0', '5.4.0', '5.5.0', '5.5.1', '5.6.0', '5.6.1') group by dt,channels limit 10000"

start=time.time()
dd=getQueryData(sql)

df=pd.read_json(dd)
end=time.time()
end-start

df.to_csv(path+'new_num'+'.csv')

In [14]:
df=pd.read_csv(path+'new_num'+'.csv',index_col=0)

In [15]:
df

,active_num,channels,dt
0,5644,NaN,2016-03-04
1,0,亚马逊应用商店,2016-03-19
2,2,亚马逊应用商店,2016-02-10
3,417,安智,2016-06-18
4,4686,升级,2016-04-01
5,0,安粉网,2016-04-01
6,2417,百度移动应用,2016-04-30
7,3586,小米应用商店,2016-04-30
8,183,乐商店,2016-04-30
9,6,营业厅安装版,2016-04-30


In [17]:
len(df['channels'].unique())

28

In [23]:
def newdf(df,channel):
    con=df['channels']==channel
    df1=df[con]
    return df1.iloc[:,[0,2]]

def newdf1(df):
    '''
    去掉单值的列：肯定有更好的写法
    '''
    df1=pd.DataFrame(index=df['dt'])
    df1['新增数']=list(df['active_num'])
    df1=df1.sort_index()
    return df1

def titledf(df,channel):
    try:
        title=channel+'_'
        title+='{}'.format(df.index.min())+'-'+'{}'.format(df.index.max())
    except:
        print('data-error-titledf:no dt')
    return title

In [13]:
from pptx.enum.chart import XL_CHART_TYPE
from pptx.util import Inches
from pptx import Presentation
from pptx.chart.data import ChartData
from pptx.util import Inches
from pptx.enum.chart import XL_TICK_MARK
from pptx.util import Pt

def df2chart_data(df):
    chart_data=ChartData()
    chart_data.categories=df.index
    for i,p in enumerate(df.columns):
        chart_data.add_series(p,df[p])
        
    return chart_data

def df_addto_shapes_chart(df,shapes,x=Inches(2), y=Inches(2), cx=Inches(6), cy=Inches(4.5),\
                          chart_type=XL_CHART_TYPE.COLUMN_CLUSTERED):
    '''
    把df加到shapes中，作为chart，位置为x, y, cx, cy
    chart类型为XL_CHART_TYPE.COLUMN_CLUSTERED
    df:data,type=DataFrame
    shapes:we add df to shapes
    '''
    
    chart_data=df2chart_data(df)
    graphic_frame=shapes.add_chart(
                                    chart_type, x, y, cx, cy, chart_data
                                    )
    
    chart=graphic_frame.chart
    return chart

def df2table(df,
            shapes,
            left = Inches(6.3),
            top = Inches(0.5),
            width = Inches(3.5),
            height = Inches(3.5)):
    '''
    df.columns=1
    '''
    rows = len(df)+1
    cols = len(df.columns)+1
    table = shapes.add_table(rows, cols, left, top, width, height).table

    # set column widths
    table.columns[0].width = Inches(1)
    table.columns[1].width = Inches(3.0)

    # write column headings
    table.cell(0, 1).text = str(df.columns[-1])

    # write body cells
    for i,s in enumerate(df.iloc[:,0]):
        table.cell(i+1,1).text = str(s)
    for i,s in enumerate(df.index):
        table.cell(i+1,0).text = str(s)

def df_to_chart_ppt(dfv,prs,title,dfv_desc=None,pct=None,pct_desc=None):
    '''
    一张ppt分成四块
    1:dfv,
    2:dfv.desccribe(),table
    3:dfvs.pct_change()
    4:dfv.pct_change().describe()
    title:hint for user
    '''
    
    slide_layout_number=6
    slide_layout = prs.slide_layouts[slide_layout_number]
    slide = prs.slides.add_slide(slide_layout)
    shapes=slide.shapes
    
    #代替标题的text
    from pptx.enum.shapes import MSO_SHAPE
    x, y, cx, cy = Inches(1), Inches(0), Inches(8), Inches(0.5)
    slide.shapes.add_shape(MSO_SHAPE.ROUNDED_RECTANGLE,x, y, cx, cy)
    slide.shapes[0].text=title
    

    #作图,各月点击
    chart=df_addto_shapes_chart(dfv,shapes,x=Inches(0), y=Inches(0.5), cx=Inches(8), cy=Inches(6),\
                              chart_type=XL_CHART_TYPE.LINE)
    
    
    #修图
    from pptx.enum.chart import XL_TICK_MARK
    from pptx.util import Pt

    category_axis = chart.category_axis
    category_axis.has_major_gridlines = False
    category_axis.has_minor_gridlines = False
    category_axis.minor_tick_mark = XL_TICK_MARK.OUTSIDE
    category_axis.tick_labels.font.italic = True
    category_axis.tick_labels.font.size = Pt(18)

    value_axis = chart.value_axis
    #value_axis.maximum_scale = 50.0
    value_axis.minor_tick_mark = XL_TICK_MARK.OUTSIDE
    value_axis.has_minor_gridlines = False
    value_axis.has_major_gridlines = False

    tick_labels = value_axis.tick_labels
    tick_labels.number_format = '#,##0'
    tick_labels.font.bold = True
    tick_labels.font.size = Pt(14)


    from pptx.dml.color import RGBColor
    from pptx.enum.chart import XL_LABEL_POSITION
    
    #add table
    if dfv_desc is not None:
        df2table(dfv_desc,
                shapes,
                left = Inches(6),
                top = Inches(0.5),
                width = Inches(3.5),
                height = Inches(3.5))
    
    #pct_change()
    if pct is not None:
        chart=df_addto_shapes_chart(pct,shapes,x=Inches(0), y=Inches(4), cx=Inches(6), cy=Inches(3.5),\
                              chart_type=XL_CHART_TYPE.LINE)
    
    #修图
    from pptx.enum.chart import XL_TICK_MARK
    from pptx.util import Pt

    category_axis = chart.category_axis
    category_axis.has_major_gridlines = False
    category_axis.has_minor_gridlines = False
    category_axis.minor_tick_mark = XL_TICK_MARK.OUTSIDE
    category_axis.tick_labels.font.italic = True
    category_axis.tick_labels.font.size = Pt(18)
    
    value_axis = chart.value_axis
    #value_axis.maximum_scale = 50.0
    value_axis.minor_tick_mark = XL_TICK_MARK.OUTSIDE
    value_axis.has_minor_gridlines = False
    value_axis.has_major_gridlines = False
    
    # add table
    if pct_desc is not None:
        df2table(pct_desc,
                shapes,
                left = Inches(6),
                top = Inches(4),
                width = Inches(3.5),
                height = Inches(3.5))   

In [26]:
channel_ls=list(df['channels'].unique())
prs=Presentation()
failedsum=0
smallnew=[]
for channel in channel_ls:
    df1=newdf(df,channel)#渠道筛选
    df2=newdf1(df1)#去单值列,index时间化
    if df2.iloc[:,0].max()>=100:
        try:
            title=titledf(df2,channel)#单页标题
            #print(title)
            df_to_chart_ppt(df2,prs,title,dfv_desc=None,pct=None,pct_desc=None)
        except Exception as e:
            print('exception:',e)
            failedsum+=1
    else:
        print(channel)
        print('最大不足100')
        smallnew.append(channel)

print(failedsum)
print(smallnew)

prs.save(path+'日新增'+'.pptx')

nan
最大不足100
亚马逊应用商店
最大不足100
安粉网
最大不足100
营业厅安装版
最大不足100
91市场
最大不足100
人人网
最大不足100
短信推广11
最大不足100
机锋市场
最大不足100
木蚂蚁
最大不足100
163应用中心
最大不足100
搜狐移动中心
最大不足100
短信推广32
最大不足100
0
[nan, '亚马逊应用商店', '安粉网', '营业厅安装版', '91市场', '人人网', '短信推广11', '机锋市场', '木蚂蚁', '163应用中心', '搜狐移动中心', '短信推广32']
